In [1]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
AutoML_path = ""
log_folder = "tpot_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_MF = AutoML_path+log_folder+'/meta_features_02272022'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022_tpot'

combination_name = 'MF+OneHot'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today


In [3]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

MF = pickle.load(open(log_MF, "rb"))
MF_keys = list(MF[list(MF.keys())[0]][0].keys())
MF_np = {}
for task in MF:
    feature = [MF[task][0][key]['value'] for key in MF_keys]
    feature = [0 if type(i) != float or math.isnan(i) else i for i in feature]
    MF_np[task] = np.array(feature)
print(len(MF_np))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))

pipeline_steps_class = [i for i in pipeline_steps_class if i[3] != "Timeout" and not np.isinf(i[3])]
print(len(pipeline_steps_class))


39
39
12863
11797


In [4]:
target_test_pred_time = {}
data_names = list(MF_np.keys())
data_embed = MF_np

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = MLPRegressor(hidden_layer_sizes=(256,128,64), learning_rate='adaptive', random_state=19)
        model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (2129, 326) y_train shape: (2129,)
X_test shape: (271, 326) y_test shape: (271,)
Threshold 0.05 MAE: 2.5689859648888995 MSE: 6.605790033935497
Time: 1.7707209587097168
X_train shape: (4352, 326) y_train shape: (4352,)
X_test shape: (271, 326) y_test shape: (271,)
Threshold 0.1 MAE: 0.30936391779175204 MSE: 0.09823959583711687
Time: 3.664294958114624
X_train shape: (7632, 326) y_train shape: (7632,)
X_test shape: (271, 326) y_test shape: (271,)
Threshold 0.15000000000000002 MAE: 0.2496184411887054 MSE: 0.06599854844580631
Time: 7.858714818954468
X_train shape: (8814, 326) y_train shape: (8814,)
X_test shape: (271, 326) y_test shape: (271,)
Threshold 0.2 MAE: 0.1435699523711839 MSE: 0.023455981431574124
Time: 9.30977177619934
X_train shape: (10228, 326) y_train shape: (10228,)
X_test shape: (271, 326) y_test shape: (271,)
Threshold 0.25 MAE: 0.039178550115028576 MSE: 0.002639685340015775
Time: 10.159272909164429
X_train shape: (10830, 326) y_train shape: (10830,)
X

X_train shape: (4379, 326) y_train shape: (4379,)
X_test shape: (314, 326) y_test shape: (314,)
Threshold 0.1 MAE: 0.2847685067648268 MSE: 0.08312559584169447
Time: 4.325993061065674
X_train shape: (8336, 326) y_train shape: (8336,)
X_test shape: (314, 326) y_test shape: (314,)
Threshold 0.15000000000000002 MAE: 0.3369202190863419 MSE: 0.11674345375453556
Time: 7.220103740692139
X_train shape: (8771, 326) y_train shape: (8771,)
X_test shape: (314, 326) y_test shape: (314,)
Threshold 0.2 MAE: 0.28632519207463664 MSE: 0.08409782996249657
Time: 7.194519996643066
X_train shape: (10185, 326) y_train shape: (10185,)
X_test shape: (314, 326) y_test shape: (314,)
Threshold 0.25 MAE: 0.18650051727088127 MSE: 0.03835058904471549
Time: 7.695058822631836
X_train shape: (10787, 326) y_train shape: (10787,)
X_test shape: (314, 326) y_test shape: (314,)
Threshold 0.3 MAE: 0.057486206153434714 MSE: 0.004995823668147434
Time: 11.013726234436035
Australian_MF+OneHot_0.3 Australian_MF+OneHot_0.35
35 35
X

Threshold 0.2 MAE: 4.73827267799775 MSE: 22.455712250273912
Time: 6.678011894226074
X_train shape: (8797, 326) y_train shape: (8797,)
X_test shape: (288, 326) y_test shape: (288,)
Threshold 0.25 MAE: 5.21740308803027 MSE: 27.228523546117046
Time: 9.545171976089478
X_train shape: (9926, 326) y_train shape: (9926,)
X_test shape: (288, 326) y_test shape: (288,)
Threshold 0.3 MAE: 2.999070456845291 MSE: 9.001631858477488
Time: 11.901948928833008
X_train shape: (10492, 326) y_train shape: (10492,)
X_test shape: (288, 326) y_test shape: (288,)
Threshold 0.35000000000000003 MAE: 3.409812159003834 MSE: 11.637285264135539
Time: 10.07532525062561
X_train shape: (10813, 326) y_train shape: (10813,)
X_test shape: (288, 326) y_test shape: (288,)
Threshold 0.4 MAE: 2.478749535735963 MSE: 6.159360788035465
Time: 15.169050931930542
X_train shape: (11156, 326) y_train shape: (11156,)
X_test shape: (288, 326) y_test shape: (288,)
Threshold 0.45 MAE: 2.7537801682496528 MSE: 7.591856195228776
Time: 8.0601

Threshold 0.2 MAE: 0.1733130809264272 MSE: 0.047707407796394104
Time: 5.105790138244629
X_train shape: (8969, 326) y_train shape: (8969,)
X_test shape: (162, 326) y_test shape: (162,)
Threshold 0.25 MAE: 0.18892039853475082 MSE: 0.061454385727192105
Time: 17.020265817642212
X_train shape: (10920, 326) y_train shape: (10920,)
X_test shape: (162, 326) y_test shape: (162,)
Threshold 0.3 MAE: 0.16858670320759545 MSE: 0.04052523012119099
Time: 8.507781982421875
X_train shape: (11401, 326) y_train shape: (11401,)
X_test shape: (162, 326) y_test shape: (162,)
Threshold 0.35000000000000003 MAE: 0.3047712080351342 MSE: 0.13503175890014257
Time: 14.673377990722656
dionis_MF+OneHot_0.35 dionis_MF+OneHot_0.4
37 37
X_train shape: (11635, 326) y_train shape: (11635,)
X_test shape: (162, 326) y_test shape: (162,)
Threshold 0.45 MAE: 0.17548135960685474 MSE: 0.04235236924950907
Time: 12.167009115219116
16 fabert
X_train shape: (1175, 326) y_train shape: (1175,)
X_test shape: (239, 326) y_test shape: (

Threshold 0.05 MAE: 0.8302617628692595 MSE: 0.6952272578834469
Time: 1.3415329456329346
X_train shape: (2850, 326) y_train shape: (2850,)
X_test shape: (302, 326) y_test shape: (302,)
Threshold 0.1 MAE: 0.9928142963789415 MSE: 0.9906580609884297
Time: 2.7678630352020264
X_train shape: (4452, 326) y_train shape: (4452,)
X_test shape: (302, 326) y_test shape: (302,)
Threshold 0.15000000000000002 MAE: 0.6970353076190725 MSE: 0.48857106436353764
Time: 4.6172099113464355
jannis_MF+OneHot_0.15 jannis_MF+OneHot_0.2
15 15
X_train shape: (6971, 326) y_train shape: (6971,)
X_test shape: (302, 326) y_test shape: (302,)
Threshold 0.25 MAE: 0.6985450063365133 MSE: 0.4910705817055424
Time: 7.0367138385772705
X_train shape: (7946, 326) y_train shape: (7946,)
X_test shape: (302, 326) y_test shape: (302,)
Threshold 0.3 MAE: 0.6182759762095509 MSE: 0.38480727684762184
Time: 10.67256498336792
X_train shape: (9715, 326) y_train shape: (9715,)
X_test shape: (302, 326) y_test shape: (302,)
Threshold 0.35000

Threshold 0.2 MAE: 0.8564461707925122 MSE: 0.7414335867124408
Time: 8.552736043930054
X_train shape: (6663, 326) y_train shape: (6663,)
X_test shape: (362, 326) y_test shape: (362,)
Threshold 0.25 MAE: 0.6241560902722997 MSE: 0.3977016168865746
Time: 7.191162109375
X_train shape: (8288, 326) y_train shape: (8288,)
X_test shape: (362, 326) y_test shape: (362,)
Threshold 0.3 MAE: 0.8204657158030366 MSE: 0.6787270693887669
Time: 7.394987106323242
X_train shape: (8723, 326) y_train shape: (8723,)
X_test shape: (362, 326) y_test shape: (362,)
Threshold 0.35000000000000003 MAE: 0.8479396164416731 MSE: 0.7232253988404859
Time: 7.452080726623535
X_train shape: (10137, 326) y_train shape: (10137,)
X_test shape: (362, 326) y_test shape: (362,)
Threshold 0.4 MAE: 0.2684092117167528 MSE: 0.07561896311686338
Time: 11.13695216178894
X_train shape: (10739, 326) y_train shape: (10739,)
X_test shape: (362, 326) y_test shape: (362,)
Threshold 0.45 MAE: 0.5638809194419464 MSE: 0.3240812049038131
Time: 9.

Threshold 0.25 MAE: 0.148746700334159 MSE: 0.024675026907859842
Time: 10.405425786972046
X_train shape: (10775, 326) y_train shape: (10775,)
X_test shape: (326, 326) y_test shape: (326,)
Threshold 0.3 MAE: 0.1485609794591775 MSE: 0.024666190262272757
Time: 13.775331735610962
X_train shape: (11118, 326) y_train shape: (11118,)
X_test shape: (326, 326) y_test shape: (326,)
Threshold 0.35000000000000003 MAE: 0.146120295018689 MSE: 0.024618704394824536
Time: 15.992784261703491
phoneme_MF+OneHot_0.35 phoneme_MF+OneHot_0.4
36 36
X_train shape: (11237, 326) y_train shape: (11237,)
X_test shape: (326, 326) y_test shape: (326,)
Threshold 0.45 MAE: 0.16819696518646413 MSE: 0.03187648597459768
Time: 13.401055097579956
32 riccardo
X_train shape: (3386, 326) y_train shape: (3386,)
X_test shape: (217, 326) y_test shape: (217,)
Threshold 0.05 MAE: 0.0631391990264733 MSE: 0.006961840834628915
Time: 2.628462076187134
X_train shape: (4073, 326) y_train shape: (4073,)
X_test shape: (217, 326) y_test shap

X_train shape: (10733, 326) y_train shape: (10733,)
X_test shape: (368, 326) y_test shape: (368,)
Threshold 0.4 MAE: 0.14987711457352818 MSE: 0.027620583601430812
Time: 13.927855730056763
X_train shape: (11076, 326) y_train shape: (11076,)
X_test shape: (368, 326) y_test shape: (368,)
Threshold 0.45 MAE: 0.16533061587498005 MSE: 0.03133834463890724
Time: 12.36202597618103
37 vehicle
X_train shape: (2016, 326) y_train shape: (2016,)
X_test shape: (379, 326) y_test shape: (379,)
Threshold 0.05 MAE: 0.28847442848760574 MSE: 0.09027518451409028
Time: 3.1173129081726074
X_train shape: (3430, 326) y_train shape: (3430,)
X_test shape: (379, 326) y_test shape: (379,)
Threshold 0.1 MAE: 0.4971226984529562 MSE: 0.25467994608914335
Time: 4.697964906692505
X_train shape: (6111, 326) y_train shape: (6111,)
X_test shape: (379, 326) y_test shape: (379,)
Threshold 0.15000000000000002 MAE: 0.1612440676507596 MSE: 0.03180071791317291
Time: 7.855648994445801
X_train shape: (6751, 326) y_train shape: (675